# Test de rapidité du modèle utilisé avec Keras (Resnet18)

Dans ce notebook, nous allons tester la rapidité du modèle lorsqu'il est utilisé avec Keras - Tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras

# Création du modèle

In [ ]:
import classification_models
from classification_models.tfkeras import Classifiers

# Chargement du modèle ResNEt18
ResNet18, preprocess_input = Classifiers.get('resnet18')

# Instanciation du modèle pré-entrainé ResNet18
base_model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=False,pooling=False)

# Désactivation des couches pour l'entrainement
for layer in base_model.layers:
    layer.trainable = False
    
# Ajout de l'applatissement des sorties et de la couche dense avec 2 neurones"
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(units=2, activation='softmax')(x)

model = tf.keras.Model(inputs=[base_model.input], outputs=[output])

On charge ensuite les poids sauvegardés lors de l'entrainement :

In [ ]:
model.load_weights("meilleur_modele_colab.hdf5")

# Utilisation du modèle

Nous allons utiliser notre modèle pour mesurer son temps de calcul et son débit pour traiter les images. Du fait de l'initialisation du GPU, nous allons mesurer ces caractéristiques après avoir utilisé le modèle un petit nombre de fois :

In [ ]:
import time

nbr_run = 100
delais = []
predictions = []

image = np.zeros((224, 224, 3), dtype=np.float32)

# Initialisation des calculs
print("Initialisation des calculs...")
for i in range(5):
    prediction = model(tf.expand_dims(image,0))
    predictions.append(float(np.asarray(prediction[0])[0]))

# Lance les inférences
for i in range(nbr_run):
    time0 = time.time()
    prediction = model(tf.expand_dims(image,0))
    time_end = time.time()
        
    delais = np.append(delais,time_end - time0)
    predictions.append(float(np.asarray(prediction[0])[0]))
        
    if i%10 == 0:
        print("Etape %d-%d moyenne : %4.1f ms" %(i,i+5,(delais[-10:].mean())*1000))
                       
print("Débit : %.0f images/s" %(nbr_run / delais.sum()))